# Bepaal het effect van een stijging van het peil van de Amstel
Dit notebook berekent het effect van een stijging van het peil van Amstel. Voor een gebied van 4 bij 3 km lezen we data in, zetten dit om naar modelinvoer en rekenen een tijdsafhankelijk grondwatermodel voor enkele jaren door.

Het notebook is slechts ter illustratie van de mogelijkheden van nlmod. Hoewel er gerekend wordt met echte data voor de ondergrond en het oppervlaktewater, wordt ontbrekende/onbekende data (bijvoorbeeld de weerstand van de Holocene deklaag, het peil van de Amstel, of de weerstand van het oppervlaktewater) opgevuld met eenvoudige aannames. Het model is op geen enkele manier gecontrolleerd of gekalibreerd, bijvoorbeeld op basis van metingen. Het model geeft de werkelijkheid waarschijnlijk dan ook niet goed weer.

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import flopy
import nlmod
logger = nlmod.util.get_color_logger("INFO");

In [ ]:
model_name = "Amstel"
model_ws = "amstel"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
extent = [118_000, 122_000, 473_000, 476_000]
line = [(118_000, 474_500), (122_000, 474_500)]
time = pd.date_range("2020", "2023", freq="MS")  # monthly timestep

## Lagenmodel
We lezen een lagenmodel in op basis van REGIS.

In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent,
    use_regis=True,
    use_geotop=False,
    cachedir=cachedir,
    cachename="layer_model.nc",
)
layer_model

Wanneer we het lagenomdel omzetten naar een model Dataset zien we aan de log-berichten dat er geen informatie is over de doorlatendheid van de Holocene deklaag, en dat nlmod deze doorlatendheden opvult met de standaard-waarden van kh=1.0 m/d, en kv=0.1 m/d.

In [ ]:
ds = nlmod.to_model_ds(layer_model, model_name, model_ws, delr=50.0)
ds = nlmod.time.set_ds_time(ds, time=time)

We kunnen het lagenmodel weergeven in een (eerder gedefinieerde) doorsnede van west naar oost. De dikte van de Holocene deklaag bedraagt ongeveer 10 meter in deze doorsnede.

In [ ]:
f, ax = plt.subplots(figsize=(13, 5))
dcs = nlmod.plot.DatasetCrossSection(ds, line, ax=ax, zmin=-100)
colors = nlmod.read.regis.get_legend()['color'].to_dict()
for layer in ds.layer.data:
    if layer not in colors:
        colors[layer] = colors['HLc']
dcs.plot_layers(colors=colors, min_label_area=1000.0)
dcs.plot_grid(vertical=False);

Oefening 1: geef de hozizontale doorlatendheid (kh) weer in een doorsnede langs dezelfde lijn. Gebruik de methode `dcs.plot_array()` om de variabele 'kh' in ds te tekenen (voeg dus de variabele `ds["kh"]` toe als eerste argument aan `dcs.plot_array()`).

Oefening 2: Geef de dikte van de holocene deklaag weer. Bereken de dikte van de lagen met behlp van de methode `nlmod.layers.calculate_thickness(ds)` en selecteer de eerste laag (`"HLc"`) uit de resulterde DataArray met behulp van `da.loc[]`.

## Download oppervlaktewaterdata
We downloaden de locatie van het oppervlaktewater uit de BasisRegistratie Grootschalige Topografie (BGT), en peilgebieden van de in het modelgebied liggende waterschappen (in dit geval alleen van Waterschap Amstel Gooi en Vecht). Om deze webservces niet te overbelasten, slaan we het resultaat op in een bestand genaamd `bgt.geojson`, zodat we dit slechts één keer doen.

In [ ]:
fname = os.path.join(cachedir, 'bgt.geojson')
if not os.path.isfile(fname):
    bgt = nlmod.read.bgt.get_bgt(extent)
    la = nlmod.gwf.surface_water.download_level_areas(bgt, extent=extent, raise_exceptions=False)
    bgt = nlmod.gwf.surface_water.add_stages_from_waterboards(bgt, la=la)
    bgt.to_file(fname, driver="GeoJSON")
else:
    logger.info('using cached data -> bgt.geojson')
bgt = gpd.read_file(fname)

Het resulterende object is een zogenaamd GeoDataFrame, waarbij elke rij een watervalk voorstelt. Het zomerpeil ('summer_stage') en winterpeil ('winter_stage') zijn afgeleid uit de peilgebieden-data en als extra kolommen door nlmod aan de bgt-data toegevoegd.

In [ ]:
bgt

Oefening 3: geef de gedownloade polygonen van het oppervlaktewater weer. Maak een kaart aan met de functie `f, ax = nlmod.plot.get_map(extent)` en plot het oppervlaktewater met `bgt.plot(ax=ax)`.

Oefening 4: Geef de bronhouder weer van de data. Voeg `"bronhouder"` toe als eerste argument aan `bgt.plot()` en geef een legenda weer door een extra argument `legend=True` mee te geven.

Uit deze figuur is te zien dat 'W0155' de brondhouder is van de meeste waterlopen in dit gebied: Waterschap Amstel, Gooi en Vecht. De Amstel zelf heeft als bronhouder de provincie Noord-Holland.

Een belangrijke invoer voor het grondwatermodel is het zomerpeil en het winterpeil. Deze geven we ruimtelijk weer.

In [ ]:
f, axes = nlmod.plot.get_map(extent, ncols=2)
norm = matplotlib.colors.Normalize(-7.0, -2.0)
cmap = 'turbo_r'

bgt.plot(color='k', ax=axes[0])
bgt.plot('summer_stage', ax=axes[0], norm=norm, cmap=cmap)
nlmod.plot.title_inside('Zomerpeil', ax=axes[0])

bgt.plot(color='k', ax=axes[1])
bgt.plot('winter_stage', ax=axes[1], norm=norm, cmap=cmap)
nlmod.plot.colorbar_inside(cmap=cmap, norm=norm)
nlmod.plot.title_inside('Winterpeil', ax=axes[1]);

Locaties die zwart blijven in beide figuren vallen niet in een peilgebied, en krijgen dan ook geen zomer- of winterpeil toegekend. Dit zijn de Amstel zelf en de aangrenzende waterloop: de Oude Waver. Voor de Amstel geven we later in dit notebook een handmatig peil op. Voor de Oude Waver laten we de peilen leeg, waardoor deze genegeerd wordt bij het aanmaken van de oppervlaktewater-invoer van het model (zie ook de log-berichten later in dit notebook).

In het noordoosten van het gebied zijn enkele locaties zonder winterpeil, waar wel een zomerpeil gedefinieerd is (wellicht veroorzaakt door een fout bij de interpretatie van de waterschapsdata door nlmod). Daarom vullen we ontbrekende waarden van het winterpeil in door het zomerpeil. De resterende ontbrekende peilen bestaan dan uit de Oude Waver. 

In [ ]:
mask = bgt["winter_stage"].isna()
bgt.loc[mask, "winter_stage"] = bgt.loc[mask, "summer_stage"]

## Download neerslag en verdamping
We downloaden neerslag en verdamping van het KNMI en berekenen hieruit de grondwateraanvulling (het neerslagoverschot) in elke tijdstap, en voegen dit toe aan de modeldataset met de vraiabele `"recharge"'.

In [ ]:
knmi_ds = nlmod.read.knmi.get_recharge(ds, cachedir=cachedir, cachename="recharge.nc")
ds.update(knmi_ds)

## Maak een Modflow 6 simulation
We genereren een Modflow 6 simulation (sim) en groundwater flow (gwf) model voor flopy aan.

In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim, complexity='moderate')

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
dis = nlmod.gwf.dis(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=0.0)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

# create storage package
sto = nlmod.gwf.sto(ds, gwf)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

## Verwerken oppervlaktewater
We vergridden de oppervlaktewater-vlakken, bereknen de conductane, en splitsen de Amstel van de andere vlakken.

In [ ]:
bgt_grid = nlmod.grid.gdf_to_grid(bgt, ds).set_index("cellid")

bed_resistance = 1.0
bgt_grid["cond"] = bgt_grid.area / bed_resistance

# handle the Amstel as a river
mask = bgt_grid["bronhouder"] == "P0027"
amstel = bgt_grid[mask]
bgt_grid = bgt_grid[~mask]

# we remove other surface water features in the same cell as the river Amstel
bgt_grid = bgt_grid[~bgt_grid.index.isin(amstel.index)]

## Voeg de Amstel toe via de RIV-package
We voegen de Amstel toe aan het model via de River-package, wat betekent dat deze ook kan infiltreren.

In [ ]:
amstel["stage"] = -0.4
amstel["rbot"] = -3.0
spd = nlmod.gwf.surface_water.build_spd(amstel, "RIV", ds)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data={0: spd})

## Voeg het overige oppervlaketwater toe via de DRN-package
We voegen al het overige oppervlaktewater toe als drains, wat betekent dat deze in het model niet kunnen infiltreren.

In [ ]:
drn = nlmod.gwf.surface_water.gdf_to_seasonal_pkg(bgt_grid, gwf, ds, save_flows=True);

## Voer het model uit en lees uitvoer in

In [ ]:
nlmod.sim.write_and_run(sim, ds)

We lezen de stijghoogtes en het drainage-debiet in, naar variabelen met namen die eindigen op "0" (ter onderscheid van een volgende berekening).

In [ ]:
head0 = nlmod.gwf.output.get_heads_da(ds)
# Get the groundwater level, which is the head in the upper active layer (not all layers are present everywhere)
gws0 = nlmod.gwf.output.get_gwl_from_wet_cells(head0)
q_drn0 = nlmod.gwf.output.get_budget_da('DRN', ds=ds)

## Teken de gemiddelde grondwaterstand
We geven de gemiddelde grondwaterstand in het model weer. Ook geven we een punt weer waar we straks een tijd-grafiek gaan weergeven

In [ ]:
f, ax = nlmod.plot.get_map(extent)
pc = nlmod.plot.data_array(gws0.mean("time"), ds=ds)
cbar = nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none")
xp = 120300
yp = 474500
ax.plot(xp, yp, marker='x', color='r');

Geef de stijghoogte in de tijd weer in de verschillende lagen bij het hiervoor gedefinieerde punt, aangegeven met een rood kruisje in voorgaande figuur.

In [ ]:
head_point = nlmod.gwf.get_head_at_point(head0, x=xp, y=yp, ds=ds).to_pandas()
ax = head_point.plot(figsize=(13,8))
ax.figure.tight_layout(pad=0.0)

Oefening 5: Geef het gemiddelde drainage-debiet van elke modelcel weer. Bereken hiertoe het drainagedebiet in mm/d, via de methode `da = -q_drn0.sum('layer') / ds['area'] * 1000`. Kijk hoe de grondwaterstand eerder werd getekend, en vervang `gws0` door `da`. En mooie kleurenschaal (`cmap`) voor deze data is `"Blues"`. Vieg daarom het extra argument `cmap="Blues"` toe aan `nlmod.plot.data_array()`.

## Reken het model nogmaals door met een hoger peil van de Amstel
We willen nu weten wat de invloed is van een hoger peil van de Amstel. Daarom zetten we het peil ('stage') van de Amstel 1 meter hoger, van -0.4 naar 0.6 m NAP, en rekenen het model opnieuw door.

In [ ]:
# remove the current RIV-package
gwf.remove_package('RIV')
# set the stage of the Amstel
amstel["stage"] = 0.6
# generate a new riv package
spd = nlmod.gwf.surface_water.build_spd(amstel, "RIV", ds)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data={0: spd})
# run the model again
nlmod.sim.write_and_run(sim, ds)

We lezen de uitvoer van model opnieuw in, naar variabelen met namen die eindigen op "1".

In [ ]:
head1 = nlmod.gwf.output.get_heads_da(ds)
gws1 = nlmod.gwf.output.get_gwl_from_wet_cells(head1)
q_drn1 = nlmod.gwf.output.get_budget_da('DRN', ds=ds)

## Geef het verschil in grondwaterstand tussen de twee modelsimulaties weer

In [ ]:
f, ax = nlmod.plot.get_map(extent)
dgws = gws1 - gws0
pc = nlmod.plot.data_array(dgws.mean("time"), ds=ds, vmin=0, vmax=1.0, cmap='magma_r')
cbar = nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none");

Het effect in de polders is dus minimaal. Wellicht neemt het drainagedebiet in de buurt van de Amstel wel toe?

Oefening 6: geef de toename van het drainagedebiet weer in mm/d.

## Extra oefeningen
Snel klaar? Niet getreurd, met onderstaande oefeningen kan je de rest van de middag vullen.

Oefening 7: Voeg gridverfijing toe rond de rivier de Amstel, door de celgrootte hier te halveren. Voeg de functie `nlmod.grid.refine()` toe na `nlmod.to_model_ds()`. Zie het notebook https://nlmod.readthedocs.io/en/stable/examples/09_schoonhoven.html voor een voorbeeld van de argumenten voor deze functie.

Oefening 8: de bovenzijde van het model is de Holocece deklaag (HLc), waarvoor REGIS geen doorlatendheid geeft. Daarom wordt kh op een standaardwaarde van 1.0 m/d en kv op 0.1 m/d gezet (zie de log-berichten na `nlmod.to_model_ds()`). We kunnen GeoTOP gebruiken om een betere schatting te krijgen voor deze doorlatendheden. Verander een argument in `nlmod.read.regis.get_combined_layer_models()` en herbereken het effect van een toename van het peil van Amstel met 1 meter. Omdat de GeoTOP-data meer variatie heeft, met hoge en lage doorlatendheden dichtbij elkaar, heeft het model moeite om te convergeren. Verander het `complexity` argument in `nlmod.sim.ims()` naar `"complex"`, zodat het model wel weer convergeert.